In [1]:
%pwd

"c:\\Users\\muska\\Desktop\\Sohel's Space\\Projects\\AI_Medical_Assistant\\Medical_AiBot\\reserch"

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

"c:\\Users\\muska\\Desktop\\Sohel's Space\\Projects\\AI_Medical_Assistant\\Medical_AiBot"

1. Data Extaction

In [5]:
# importing mdoules to load pdf files
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

In [7]:
def get_pdf(data):
    loader= DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader,
    )

    document= loader.load()

    print(f"Number of documents loaded: {len(document)}")
    return document

In [8]:
docs= get_pdf("data")

Number of documents loaded: 637


In [9]:
docs[0].metadata

{'producer': 'PDFlib+PDI 5.0.0 (SunOS)',
 'creator': 'PyPDF',
 'creationdate': '2004-12-18T17:00:02-05:00',
 'moddate': '2004-12-18T16:15:31-06:00',
 'source': 'data\\Medical_book.pdf',
 'total_pages': 637,
 'page': 0,
 'page_label': '1'}

In [10]:
from langchain.schema import Document

"""function to clean metadata of documents"""

def clean_docs(docs):
    cleaned_docs= []

    for doc in docs:
        metadata= doc.metadata["source"]

        cleaned_docs.append(
            Document(
                page_content= doc.page_content,
                metadata= {"source": metadata}
            )
        )
    return cleaned_docs

In [11]:
clean_docs= clean_docs(docs)

In [12]:
clean_docs[10].metadata

{'source': 'data\\Medical_book.pdf'}

2. Text Splitting/Chunking

In [13]:
# importing langchain's text splitter module
"""This module provides functionality to split documents into smaller chunks."""
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [14]:
"""function to split documents into smaller chunks"""
def split_docs(clean_docs):
    text_splitter= RecursiveCharacterTextSplitter(
        chunk_size= 500,
        chunk_overlap= 20,
    )

    splitted_docs= text_splitter.split_documents(clean_docs)

    print(f"Number of split documents: {len(splitted_docs)}")
    return splitted_docs

In [15]:
splitted_docs= split_docs(clean_docs)

Number of split documents: 5859


3. Embeddings

In [16]:
# Import embedding model
from langchain.embeddings import HuggingFaceBgeEmbeddings


In [17]:
"""
function to get embedding model.
This function initializes and returns a HuggingFaceBgeEmbeddings model using the specified model name.
"""

def get_embeddings():
    model_name= "sentence-transformers/all-MiniLM-L6-v2"
    embedding_model= HuggingFaceBgeEmbeddings(
        model_name= model_name
        )

    return embedding_model

embedding_model= get_embeddings()

C:\Users\muska\AppData\Local\Temp\ipykernel_7092\185403826.py:8: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model= HuggingFaceBgeEmbeddings(


In [18]:
embedding_model

HuggingFaceBgeEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_instruction='Represent this question for searching relevant passages: ', embed_instruction='', show_progress=False)

In [19]:
vector= embedding_model.embed_query("Hello world how are you, I am sohel Mulani")

In [19]:
vector

[0.028469542041420937,
 0.11501668393611908,
 0.03270135074853897,
 0.007951495237648487,
 -0.0065203639678657055,
 -0.04722017049789429,
 0.06523384153842926,
 -0.033700983971357346,
 -0.041105564683675766,
 -0.04156296327710152,
 0.07509908080101013,
 -0.1089206263422966,
 -0.009411098435521126,
 -0.05619879439473152,
 0.0847795233130455,
 -0.009853297844529152,
 0.03278208523988724,
 -0.03715991973876953,
 -0.11123354732990265,
 -0.019586894661188126,
 0.08424617350101471,
 0.08642211556434631,
 0.0049339886754751205,
 0.0642814189195633,
 -0.05126078799366951,
 -0.027468116953969002,
 0.019231485202908516,
 -0.039615798741579056,
 0.07078070193529129,
 -0.01674046739935875,
 -0.07029733061790466,
 0.008218610659241676,
 0.09632854163646698,
 0.03770509734749794,
 -0.0495634600520134,
 0.14160031080245972,
 0.04099220409989357,
 -0.026685262098908424,
 -0.01357988454401493,
 -0.03946810960769653,
 -0.0065088942646980286,
 -0.0482693575322628,
 0.005567948799580336,
 -0.0511361435055

In [20]:

print(len(vector))

384


4. Storing vectors to vectorDB

In [21]:
#getting environment variables
import os
from dotenv import load_dotenv

load_dotenv()

PINECONE_API_KEY= os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY= os.getenv("OPENAI_API_KEY")


In [22]:
# importing pinecone module
"""
This module provides functionality to interact with the Pinecone vector database.
Pinecone is a managed vector database service that enables efficient storage and 
retrieval of high-dimensional vector data. 
It is commonly used in applications such as machine learning, natural language processing, 
and recommendation systems to handle large-scale vector embeddings.

ServerlessSpec: A class that defines the specifications for creating a serverless Pinecone index.
Pinecone: The main class for interacting with the Pinecone service, including creating and managing indexes.

"""
from pinecone import Pinecone, ServerlessSpec

In [23]:
# create pinecone index and initialize pinecone client

# define index name
index_name= "medical-assistant"

# initialize pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# define serverless specification
spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
    )

# create index if it does not exist
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name, 
        dimension=384, 
        metric="cosine", 
        spec=spec
        )

# connect to the index  
index= pc.Index(index_name)



In [24]:
#import langchain pinecone
from langchain_pinecone import PineconeVectorStore

doc_vector_store= PineconeVectorStore.from_documents(
    index_name= index_name,
    embedding= embedding_model,
    documents= splitted_docs
    )



In [25]:
#Adding more documents to pinecone vector store
extra_doc= Document(
    page_content= "This is a sample document to add more documents to pinecone vector store.",
    metadata= {"source": "extra_doc.pdf"}
)
doc_vector_store.add_documents([extra_doc])

['89728379-af00-4273-8078-7bafd8fa2aa3']

5. Creating a retriever from the pinecone vector store

In [26]:
retriver= doc_vector_store.as_retriever(search_type="similarity", search_kwargs={"k":3})
retriveed_docs= retriver.invoke("What is diabetes?")
retriveed_docs

[Document(id='378ec838-1484-447f-8f5a-343b4284aa87', metadata={'source': 'data\\Medical_book.pdf'}, page_content='begin to fall. A person with diabetes mellitus either does\nnot make enough insulin, or makes insulin that does not\nwork properly. The result is blood sugar that remains\nhigh, a condition called hyperglycemia.\nDiabetes must be diagnosed as early as possible. If\nleft untreated, it can damage or cause failure of the eyes,\nkidneys, nerves, heart, blood vessels, and other body\norgans. Hypoglycemia, or low blood sugar, may also be\ndiscovered through blood sugar testing. Hypoglycemia is'),
 Document(id='a37486e9-37a6-4bd3-a465-81fd8779db17', metadata={'source': 'data\\Medical_book.pdf'}, page_content='begin to fall. A person with diabetes mellitus either does\nnot make enough insulin, or makes insulin that does not\nwork properly. The result is blood sugar that remains\nhigh, a condition called hyperglycemia.\nDiabetes must be diagnosed as early as possible. If\nleft untre

6. Create LLM model

In [27]:
# creating OpenAI LLM model for question answering
from langchain_openai import ChatOpenAI
llm_chatModel= ChatOpenAI(
    openai_api_key= OPENAI_API_KEY,
    model_name= "gpt-5-nano"
    )

In [28]:
#import chains module for question answering

"""
langchain.chains: This module provides functionality to create and manage chains of operations for various tasks,
including retrieval-based question answering.

langchain.chains.create_retrieval_chain: A function that creates a retrieval-based question-answering chain.

langchain.chains.combine_documents.create_stuff_documents_chain: A function that creates a document combination chain using the "stuff" method.

langchain_core.prompts.ChatPromptTemplate: A class that defines a chat prompt template for use in chat-based language models.
"""
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


In [ ]:
# defining system prompt for question answering
system_prompt= (
    "You are a helpful medical assistant for question answering tasks. Use the following context to answer the user's question."
    "If you don't know the answer, just say that you don't know, don't try to make up an answer."
    "Keep your answers short, concise and to the point."
    "\n\n"
    "{context}"
)

# create prompt template using system prompt 
"""
Prompt template for question answering tasks using chat-based language models.
ChatPromptTemplate.from_messages: A method that creates a chat prompt template from a list of messages, including system and user messages.
Here, the system message contains the system prompt, and the user message is a placeholder for user input.
input: The user's question or input that will be used in the prompt template.
"""
prompt_template= ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}"),
    ]
    )

In [ ]:
#create retrieval chain for question answering
# LLM pipelineb
"""
This code creates a retrieval-based question-answering chain using a language model and a prompt template.
create_stuff_documents_chain: A function that creates a document combination chain using the "stuff" method.
llm: The language model to be used for generating answers.
prompt: The prompt template to be used for formatting the input and context.
create_retrieval_chain: A function that creates a retrieval-based question-answering chain.
"""
retrieval_chain= create_stuff_documents_chain(
    llm= llm_chatModel,
    prompt= prompt_template
    )

rag_chain= create_retrieval_chain(retriver, retrieval_chain)



In [ ]:
# ask questions to the RAG chain

"""
invoke: A method that executes the retrieval-augmented generation (RAG) chain with the provided input.
input: The user's question or input to be answered by the RAG chain.
"""
query= "what is acremogaly?"
response= rag_chain.invoke({"input": query})
print(response["answer"])

You probably mean acromegaly. It’s a hormonal disorder caused by too much growth hormone, usually from a pituitary tumor, after puberty. This leads to enlargement of hands, feet, and facial features, thickened skin, and joint problems; it can also cause diabetes, high blood pressure, and heart issues. In children, excess GH causes gigantism. Diagnosis involves measuring IGF-1, a GH suppression test, and pituitary MRI. Treatment typically includes removing the tumor (surgery), medications to block GH effects, and sometimes radiation.


In [41]:
query= "what is treatment of acremogaly?"
response= rag_chain.invoke({"input": query})
print(response["answer"])

Acromegaly is treated mainly to normalize GH/IGF-1 levels and control the pituitary tumor. Options include:

- Primary therapy: transsphenoidal surgery to remove the pituitary adenoma (often curative if the tumor is small).

- Medical therapy (if surgery isn’t possible or not curative):
  - Somatostatin analogs (octreotide, lanreotide) to suppress GH/IGF-1.
  - GH receptor antagonist (pegvisomant) to normalize IGF-1.
  - Dopamine agonists (cabergoline, bromocriptine) in selected cases with mild disease.

- Radiotherapy: for residual or recurrent tumor or when other treatments fail.

- Management of comorbidities (diabetes, hypertension, sleep apnea) and regular monitoring of IGF-1 and pituitary function.
